# Import libraries

In [9]:
# matplotlib plotting
import matplotlib as mpl
from matplotlib import colors
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
# images
import cv2 # OpenCV
from PIL import Image
# outlier detection
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
# clustering
from sklearn.cluster import *
# numpy
import numpy as np

# Define initial variables

* **title**: video file name
* **feature_params**: params for feature extraction
* **lk_params**: params for Lucas–Kanade method
* **mem**: memory size, how long features will be tracked
* ***view params***:
     * **distr**: view type. If 0, view vectors and position; if 1, view only length  and direction
     * **dits**: if 0, view vectors as lines, if 1, view vectors as arrows
     * **cluster**: if 0, disable clustering, if 1, enable clustering
     * **outlier**: if 0, process all data, if 1, exclude outliers
     * **bg**: only for distr=0, if 1, show image on background, if 0, show only vectors
* ***outlier params***
     * ***Isolation Forest***
         * **n_trees**: number of trees in isolation forest
         * **cont**: contamination of isolation forest
     * ***One Class SVM***
         * **nu**: nu of One Class SVM
         * **gama**: gamma of One Class SVM
         * **kearnel**: SVM kernal; best is "rbf"
* ***Clustering param***
    * **eps**: epsilon param of DBSCAN
    * **l_lize**: number of examples in single leaf

In [3]:
#params
title = "flow300.mp4"
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 10000,
                       qualityLevel = 0.01,
                       minDistance = 8,
                       blockSize = 8 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (30,30),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.03))
#memory length
mem = 10
#view params
distr = 0
dots = 1
cluster = 1
outlier = 1
bg = 1


#outlier params
n_trees = 50
cont = 0.5
nu = 0.5
gama = 0.5
kernel = "rbf"

# clustering param
eps = 70
l_size = 10


In [8]:
#color settings
cmap_hm = [[255-i,0,i] for i in range(0,512,5)]
cmap_cl = [[255,0,0],
           [0,255,0],
           [0,0,255],
           [255,255,0],
           [255,0,255],
           [0,255,255],
           [0,128,255],
           [0,255,128],
           [255,0,128],
           [128,0,255],
           [128,0,128],
           [0,128,128],
           [128,128,0],
           [255,128,0],
           [128,255,0]]

# Define some support functions

In [6]:
#additional functions
leng = lambda x, y: np.sqrt((x[0]-y[0])*(x[0]-y[0])+(x[1]-y[1])*(x[1]-y[1]))
col = lambda x: cmap_hm[int(x)]
col_cls = lambda x: cmap_cl[x+1]

In [10]:
cap = cv2.VideoCapture(title)
# outlier detection method
if outlier:
    #outl = IsolationForest(n_estimators= n_trees, contamination= cont, n_jobs=4)
    outl = OneClassSVM(kernel=kernel, nu=nu, gamma=gama)
# clustering
if cluster:
    cls = DBSCAN(eps=eps, leaf_size=l_size, n_jobs = -1, algorithm='kd_tree')
    #cls = MeanShift(n_jobs=4)
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
masks = [np.zeros_like(old_frame) for i in range(mem)] 
vecs = [np.zeros((1,4)) for i in range(mem)]
try:
    while(1):
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        masks.append(np.zeros_like(old_frame))
        vecs.append(np.zeros((1,4)))
        for i in range(len(vecs)):
            vecs[i] = np.vstack([vecs[i], np.hstack([good_new-good_old, good_old])]) 
                                # + frame_gray[good_new[:,1].astype(int), good_new[:,0].astype(int)]
        vec = vecs.pop(0)
        if outlier:
            outl.fit(vec)
            pred = outl.predict(vec)
        else:
            pred = -1*np.ones(vec.shape[0])
        if cluster:
            clus = cls.fit_predict(vec) # FOR DBSCAN ONLY!!!
            #cls.fit(vec)
            #clus = cls.predict(np.hstack([good_new-good_old, good_old]))
        if distr:
            for i,(new,old) in enumerate(zip(good_new,good_old)):
                a,b = new.ravel()
                c,d = old.ravel()
                ln=leng((a,b), (c,d))
                if 4<ln<50 and (not outlier or pred[i] == -1):
                    if cluster:
                        co = col_cls(clus[i])
                    else:
                        co = col(ln)
                    for j in masks:
                        if dots:
                            j = cv2.circle(j,(int(400+(a-c)*5), int(300+(b-d)*5)),2,co,-1)
                        else:
                            j = cv2.line(j, (400,300),(int(400+(a-c)*5), int(300+(b-d)*5)),co, 1)
        else:
            for i,(new,old) in enumerate(zip(good_new,good_old)):
                a,b = new.ravel()
                c,d = old.ravel()
                ln=leng((a,b), (c,d))
                if 4<ln<50 and (not outlier or pred[i] == -1):
                    if cluster:
                        co = col_cls(clus[i])
                    else:
                        co = col(ln)
                    for j in masks:
                        j = cv2.line(j, (a,b),(c,d), co, 1)
                        if dots:
                            j = cv2.circle(j,(a,b),1,co,-1)
        mask = masks.pop(0)
        if not distr and bg:
            img = cv2.add(255-frame,mask)
        else:
            img = mask

        cv2.imshow('frame',img)
        k = cv2.waitKey(100) & 0xff
        if k == 27:
            break

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = np.vstack([good_new.reshape(-1,1,2),cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)])[-100:,:,:]
except Exception as e:
    print(e)
cv2.destroyAllWindows()
cap.release()

D:\Build\OpenCV\opencv-3.2.0\modules\imgproc\src\color.cpp:9748: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor

